<a href="https://www.kaggle.com/code/jibonk/titanic-logistic-regrssion?scriptVersionId=242875075" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 
import warnings


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


warnings.filterwarnings('ignore')

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
577,578,1,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
748,749,0,1,"Marvin, Mr. Daniel Warner",male,19.0,1,0,113773,53.1000,D30,S
289,290,1,3,"Connolly, Miss. Kate",female,22.0,0,0,370373,7.7500,NaN,Q
102,103,0,1,"White, Mr. Richard Frasar",male,21.0,0,1,35281,77.2875,D26,S


In [3]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test_pids = test["PassengerId"]

In [4]:
def clean(data):
    data = data.drop(["Ticket", "PassengerId", "Name", "Cabin"], axis=1)
    
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data['Embarked'].fillna("U", inplace=True)
    return data

train = clean(train)
test = clean(test)

In [5]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
cols = ["Sex", "Embarked"]

for col in cols:
    train[col] = label_encoder.fit_transform(train[col])
    test[col] = label_encoder.fit_transform(test[col])
    print(label_encoder.classes_)

train.sample(5)

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
630,1,1,1,80.0,0,0,30.0000,2
173,0,3,1,21.0,0,0,7.9250,2
838,1,3,1,32.0,0,0,56.4958,2
716,1,1,0,38.0,0,0,227.5250,0
473,1,2,0,23.0,0,0,13.7917,0


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train["Survived"]
X = train.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
lr = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [8]:
print(lr.intercept_)
print(lr.coef_)

[4.54875792]
[[-9.31174408e-01 -2.59846094e+00 -3.04084790e-02 -2.94601858e-01
  -1.12477657e-01  2.56243595e-03 -2.12666413e-01]]


In [9]:
predictions = lr.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

In [10]:
submission = lr.predict(test)

In [11]:
submission_df = pd.DataFrame({"PassengerId": test_pids.values,
                   "Survived": submission,
                  })

In [12]:
submission_df.to_csv("/kaggle/working/submission.csv", index=False)